# Topic Modeling with Gensim (Python)

1. Set up Pyspark in Colab
2. Install nltk, Download nltk stopwords
3. Stemming and Lemmatization
    * Install Gensim
    * Install spaCy
    * Install other dependencies
    * Download and import speeches
4. LDA analysis
    * Data Cleaning
    * Bigrams and Trigrams
    * Topic Modeling Inputs
    * Optimal Number of Topics
    * Viewing LDA topics
5. LDA Evaluation
    * Model Perplexity and Coherence
    * Install pyLDAvis
    * Install LDA Mallet
6. Interpreting LDA Results
    * Dominant topic in each sentence
    * Most representative document for each topic
    * Topic distribution across documents
7. Conclusions
    * Strengths of Topic Modeling
    * Words of caution



## Set up Pyspark in Colab

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system.  
Follow the steps to install the dependencies:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Can take as much as 6 minutes!

!wget -qN https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop3.2"

In [ ]:
!pip install -q findspark

## 2. Install nltk, download nltk stopwords

Definition: high-frequency words like *the, to, and, also* that we sometimes want to filter out of a document before further processing.


In [ ]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Let's import the stopwords and make them available in `stop_words`.

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# Additional words we would like to ignore, for example, for a corpus of emails
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
print(len(stop_words), stop_words)

179 ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

## 3. Stemming and lemmatization

For grammatical reasons, documents are going to use different forms of a word, such as organize, organizes, and organizing. Additionally, there are families of derivationally related words with similar meanings, such as democracy, democratic, and democratization. In many situations, it seems as if it would be useful for a search for one of these words to return documents that contain another word in the set.

The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. 

For grammatical reasons, documents are going to use different forms of a word, such as organize, organizes, and organizing. Additionally, there are families of derivationally related words with similar meanings, such as democracy, democratic, and democratization. In many situations, it seems as if it would be useful for a search for one of these words to return documents that contain another word in the set.

The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. For instance:

**am, are, is $\Rightarrow$ be**

**car, cars, car's, cars' $\Rightarrow$ car**

The result of this mapping of text will be something like:

**the boy's cars are different colors $\Rightarrow$ the boy car be differ color**

* *Stemming* usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving this goal correctly most of the time, and often includes the removal of derivational affixes. For example, **saw** $\Rightarrow$ **s**.

* *Lemmatization* usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma . For example, **saw**  ⇒ **see** or **saw**  ⇒ **saw** depending on whether the use of the token was as a verb or a noun. 

Stemming is fast and crude whereas lemmatization commonly only collapses the different inflectional forms of a lemma. 

Stemming or lemmatization is often done by an additional plug-in component to the indexing process, and a number of such components exist, both commercial and open-source.

Source: [Stanford NLP](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)

In [ ]:
# installing other dependencies
import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)
import numpy as np

import re
import string
import pprint

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 4. Import Inaugural Speeches

Downloads and prepares a dictionary with

* Keys = speech_name, e.g., `1789-Washington.txt` and
* Value = List of list of sentences, e.g., [<br/>['Fellow', '-', 'Citizens', 'of', 'the', 'Senate', 'and', 'of', 'the', 'House', 'of', 'Representatives', ':'], <br/>['Among', 'the', 'vicissitudes', 'incident', 'to', 'life', 'no', 'event', 'could', 'have', 'filled', 'me', 'with', 'greater', 'anxieties', 'than', 'that', 'of', 'which', 'the', 'notification', 'was', 'transmitted', 'by', 'your', 'order', ',', 'and', 'received', 'on', 'the', '14th', 'day', 'of', 'the', 'present', 'month', '.'], <br/>...]

In [ ]:
nltk.download('inaugural')
nltk.download('punkt')

from nltk.corpus import inaugural

[nltk_data] Downloading package inaugural to /root/nltk_data...
[nltk_data]   Unzipping corpora/inaugural.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
speechesDict = {fileid: inaugural.sents(fileid) for fileid in nltk.corpus.inaugural.fileids()}
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(speechesDict)

{ '1789-Washington.txt': [['Fellow', '-', 'Citizens', 'of', 'the', 'Senate', 'and', 'of', 'the', 'House', 'of', 'Representatives', ':'], ['Among', 'the', 'vicissitudes', 'incident', 'to', 'life', 'no', 'event', 'could', 'have', 'filled', 'me', 'with', 'greater', 'anxieties', 'than', 'that', 'of', 'which', 'the', 'notification', 'was', 'transmitted', 'by', 'your', 'order', ',', 'and', 'received', 'on', 'the', '14th', 'day', 'of', 'the', 'present', 'month', '.'], ...],
  '1793-Washington.txt': [['Fellow', 'citizens', ',', 'I', 'am', 'again', 'called', 'upon', 'by', 'the', 'voice', 'of', 'my', 'country', 'to', 'execute', 'the', 'functions', 'of', 'its', 'Chief', 'Magistrate', '.'], ['When', 'the', 'occasion', 'proper', 'for', 'it', 'shall', 'arrive', ',', 'I', 'shall', 'endeavor', 'to', 'express', 'the', 'high', 'sense', 'I', 'entertain', 'of', 'this', 'distinguished', 'honor', ',', 'and', 'of', 'the', 'confidence', 'which', 'has', 'been', 'reposed', 'in', 'me', 'by', 'the', 'people', 'of

### 3. Install Gensim and spaCy model (for lemmatization)
Lemmatization is nothing but converting a word to its root word. For example: the lemma of the word ‘machines’ is ‘machine’. Likewise, ‘walking’ –> ‘walk’, ‘mice’ –> ‘mouse’ and so on.

In [ ]:
!pip install gensim
!python3 -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-14 17:31:55.363238: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 17:31:56.995071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 96.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# def collect (sentences):
#     speech = []
#     for sent in sentences:
#         for fragment in sent:
#             speech.extend(fragment)
#     return speech
# df['content'] = df.apply(lambda row: collect(row[1:]), axis=1)

In [ ]:
# Gensim

import gensim
from gensim import corpora

from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
spacy.load('en_core_web_sm')

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

### 4. Load Speeches

In [ ]:
speechesDict = {fileid: inaugural.sents(fileid) for fileid in nltk.corpus.inaugural.fileids()}
speechesDF = pd.DataFrame(speechesDict.items(), columns=['Filename', 'Speech'])
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

def collect (sentences):
    speech = []
    for sent in sentences:
        for fragment in sent:
            speech.extend(fragment)
    return speech
speechesDF['content'] = speechesDF.apply(lambda row: collect(row[1:]), axis=1)
speechesDF['contentLen'] = speechesDF.apply(lambda row: len(collect(row[1:])), axis=1)
# speechesDF['contentLen'] = speechesDF.apply(lambda row: len(collect(row[1:])), axis=1)
#speechesDF['speechLen'] = speechesDF.apply(lambda row: len(collect(row[1:])), axis=1)
# df['content'] = df.apply(lambda row: list(sent_to_words(row[1:])), axis=1)
# df
# df = df[["Filename","contentLen"]]#,"speechLen"]] # select multiple columns
speechesDF[["Filename", "content", "contentLen"]]

,Filename,content,contentLen
0,1789-Washington.txt,"[Fellow, -, Citizens, of, the, Senate, and, of...",8719
1,1793-Washington.txt,"[Fellow, citizens, ,, I, am, again, called, up...",799
2,1797-Adams.txt,"[When, it, was, first, perceived, ,, in, early...",14130
3,1801-Jefferson.txt,"[Friends, and, Fellow, Citizens, :, Called, up...",10333
4,1805-Jefferson.txt,"[Proceeding, ,, fellow, citizens, ,, to, that,...",13111
5,1809-Madison.txt,"[Unwilling, to, depart, from, examples, of, th...",7083
6,1813-Madison.txt,"[About, to, add, the, solemnity, of, an, oath,...",7241
7,1817-Monroe.txt,"[I, should, be, destitute, of, feeling, if, I,...",20187
8,1821-Monroe.txt,"[Fellow, citizens, ,, I, shall, not, attempt, ...",26739
9,1825-Adams.txt,"[In, compliance, with, an, usage, coeval, with...",17959


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 7. Data Cleaning</h2>
</div>


In [ ]:
dfSelectedSpeeches = speechesDF.loc[speechesDF['Filename'].isin(list(['1981-Reagan.txt', '2021-Biden.txt','1985-Reagan.txt', ]))]
dfSelectedSpeeches
for speech in ['1981-Reagan.txt', '1985-Reagan.txt', '2021-Biden.txt']:
    content = speechesDF[speechesDF['Filename'] == speech].content[:1]
    contentList = content.to_list()
    print(contentList[0])

['Senator', 'Hatfield', ',', 'Mr', '.', 'Chief', 'Justice', ',', 'Mr', '.', 'President', ',', 'Vice', 'President', 'Bush', ',', 'Vice', 'President', 'Mondale', ',', 'Senator', 'Baker', ',', 'Speaker', 'O', "'", 'Neill', ',', 'Reverend', 'Moomaw', ',', 'and', 'my', 'fellow', 'citizens', ':', 'To', 'a', 'few', 'of', 'us', 'here', 'today', ',', 'this', 'is', 'a', 'solemn', 'and', 'most', 'momentous', 'occasion', ';', 'and', 'yet', ',', 'in', 'the', 'history', 'of', 'our', 'Nation', ',', 'it', 'is', 'a', 'commonplace', 'occurrence', '.', 'The', 'orderly', 'transfer', 'of', 'authority', 'as', 'called', 'for', 'in', 'the', 'Constitution', 'routinely', 'takes', 'place', 'as', 'it', 'has', 'for', 'almost', 'two', 'centuries', 'and', 'few', 'of', 'us', 'stop', 'to', 'think', 'how', 'unique', 'we', 'really', 'are', '.', 'In', 'the', 'eyes', 'of', 'many', 'in', 'the', 'world', ',', 'this', 'every', '-', '4', '-', 'year', 'ceremony', 'we', 'accept', 'as', 'normal', 'is', 'nothing', 'less', 'than',

In [ ]:
# # dfSelectedSpeeches.groupby("content").agg(pd.Series.tolist)
# words = []
# for speech in dfSelectedSpeeches['content'].tolist():
#     # print (len(words), len(speech))
#     words.extend(speech)
#     # print (len(words))
# len(words)
# words[5740:5760]

In [ ]:
def clean_sents(data):
    # # Convert to list
    # data = df.content.values.tolist()

    # Remove Emails, not needed here
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data[0]]
    
    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]
    
    # Remove distracting single quotes
    data = [re.sub("\'"," ", sent) for sent in data]
    print(data)

    # Remove words 3 characters or less
    # Remove punctuations

    return data

speeches = speechesDF['Filename']
speeches = ['1981-Reagan.txt', '1985-Reagan.txt', '2021-Biden.txt']

# Analyze the last 12 inaugural speeches
speeches = speechesDF['Filename'][-12:]
print(speeches)
def speechify(speeches):
    words = []
    for speech in speeches:
        # print(speech)
        content = speechesDF[speechesDF['Filename'] == speech].content[:2]
        # print(content)
        contentList = content.to_list()
        # print(contentList)
        words.extend(contentList[0])
        # print(speech, len(words), words)
    return words
# len(speechify(speeches))
data = speechify(speeches)

47     1977-Carter.txt
48     1981-Reagan.txt
49     1985-Reagan.txt
50       1989-Bush.txt
51    1993-Clinton.txt
52    1997-Clinton.txt
53       2001-Bush.txt
54       2005-Bush.txt
55      2009-Obama.txt
56      2013-Obama.txt
57      2017-Trump.txt
58      2021-Biden.txt
Name: Filename, dtype: object


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 8. Tokenize words and Clean-up text </h2>
</div>

The sentences look better now, but you want to tokenize each sentence into a list of words, removing punctuations and unneccessary characters altogether.

Gensim's `simple_preprocess()` is great for this. Additionally I have set `deacc=True` to remove the punctuations.

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
data_words = [dw for dw in data_words if len(dw)>0]
print(data_words[3401:3406])


[['battalions'], ['under'], ['heavy'], ['artillery'], ['fire']]


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 9. Create Bigram and Trigram Models</h2>
</div>

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words that occur frequently.

Some examples in our example are: 'soviet union', etc.

Gensim's `Phrases` model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to `Phrases` are `min_count` and `threshold`. The higher the values of these param, the harder it is for words to be combined to bigrams.

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['for']


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 10. Remove Stopwords, Make Bigrams and Lemmatize</h2>
</div>


The bigrams model is ready. Let's define the functions to remove the stopwords, make bigrams and lemmatization and call them sequentially.

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Let’s call the functions in order.

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
print(data_words_nostops)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
print(data_words_bigrams)

# In the end, we didn't create trigrams. Should have taken the extra time.

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[[], [], [], [], [], ['nation'], ['want'], [], ['thank'], [], ['predecessor'], [], [], [], [], ['done'], [], ['heal'], [], ['land'], [], [], ['outward'], [], ['physical'], ['ceremony'], [], ['attest'], [], [], [], [], ['inner'], [], ['spiritual'], ['strength'], [], [], ['nation'], [], [], ['high'], ['school'], ['teacher'], ['miss'], ['julia'], ['coleman'], ['used'], [], ['say'], [], ['must'], ['adjust'], [], ['changing'], ['times'], [], ['still'], ['hold'], [], ['unchanging'], ['principles'], [], [], [], [], [], ['bible'], ['used'], [], [], ['inauguration'], [], [], ['first'], ['president'], [], [], [], [], ['taken'], [], ['oath'], [], ['office'], [], [], ['bible'], [], ['mother'], ['gave'], [], [], ['years'], ['ago'], ['opened'], [], ['timeless'], ['admonition'], [], [], ['ancient'], ['prophet'], ['micah'], [], ['hath'], ['showed'], ['thee'], ['man'], [], [], ['good'], [], [], ['doth'], [], ['lord'], ['require'], [], ['thee'], [], [], [], ['justly'], [], [], ['love'], ['mercy'], [], [

In [ ]:
# pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 11. Create the Dictionary and Corpus needed for Topic Modeling</h2>
</div>

The two main inputs to the LDA topic model are the dictionary(`id2word`) and the corpus. Let's create them.

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# Human readable format of corpus (term-frequency)
# There is nothing magical about 2500:2510, I just wanted to examine a random location
print ([[(id2word[id], freq) for id, freq in cp] for cp in corpus[2500:2512]])

[[], [], [], [('reflect', 1)], [], [('compassion', 1)], [], [], [], [('much', 1)], [('part', 1)], []]


Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

This is used as the input by the LDA model.


If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

Or, you can see a human readable form of the corpus itself.

In [ ]:
speeches_corpus = dict(id2word)

We are ready to build the topic model.

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 12. Building the Topic Model</h2>
</div>

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well. 

Apart from that, `alpha` and `beta` are hyperparameters that affect sparsity of the topics. According to the gensim docs, both default to 1.0/num_topics prior.

`chunksize` is the number of documents to be used in each training chunk.  `update_every` determines how often the model parameters should be updated and `passes` is the total number of training passes.

The next cell can take about 5 minutes to process.

In [ ]:
# Build LDA model
num_topics = 10
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 13. View the topics in LDA model</h2>
</div>

The above LDA model is built with 5 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using `lda_model.print_topics()` as shown next.

In [ ]:
# Print the Keywords in the topics
# pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
!pip install pyLDAvis==3.2.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.1-py2.py3-none-any.whl size=136189 sha256=1e0800afde421848e7f54777da304d70834f93ad31bbb9040cab098f03cecdef
  Stored in directory: /root/.cache/pip/wheels/d2/cc/56/70e7bd4fdf983a26b148dffa850bfb42707601f27d915f5485
Successfully built pyLDAvis


In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [ ]:
print ([itm for itm in dir(doc_lda) if not itm.startswith('__')])
print ([itm for itm in dir(doc_lda.obj) if (not itm.startswith('__')) and (not itm.startswith('_'))])
doc_lda.obj.print_topics(num_topics=-1)

['_adapt_by_suffix', '_load_specials', '_save_specials', '_smart_save', 'add_lifecycle_event', 'chunksize', 'corpus', 'load', 'metadata', 'obj', 'save', 'save_corpus']
['add_lifecycle_event', 'alpha', 'bound', 'callbacks', 'chunksize', 'clear', 'decay', 'diff', 'dispatcher', 'distributed', 'do_estep', 'do_mstep', 'dtype', 'eta', 'eval_every', 'expElogbeta', 'gamma_threshold', 'get_document_topics', 'get_term_topics', 'get_topic_terms', 'get_topics', 'id2word', 'inference', 'init_dir_prior', 'iterations', 'lifecycle_events', 'load', 'log_perplexity', 'minimum_phi_value', 'minimum_probability', 'num_terms', 'num_topics', 'num_updates', 'numworkers', 'offset', 'optimize_alpha', 'optimize_eta', 'passes', 'per_word_topics', 'print_topic', 'print_topics', 'random_state', 'save', 'show_topic', 'show_topics', 'state', 'sync_state', 'top_topics', 'update', 'update_alpha', 'update_eta', 'update_every']


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.291*"thank" + 0.075*"make" + 0.059*"century" + 0.052*"strength" + 0.050*"first" + 0.017*"freedom" + 0.007*"do" + 0.001*"high" + 0.000*"troop" + 0.000*"silent"'),
 (1,
  '0.080*"take" + 0.005*"ceremony" + 0.000*"troop" + 0.000*"object" + 0.000*"lie" + 0.000*"silent" + 0.000*"prayer" + 0.000*"shoe" + 0.000*"beacon" + 0.000*"forebearer"'),
 (2,
  '0.202*"time" + 0.000*"troop" + 0.000*"silent" + 0.000*"object" + 0.000*"lie" + 0.000*"prayer" + 0.000*"shoe" + 0.000*"beacon" + 0.000*"owe" + 0.000*"devote"'),
 (3,
  '0.000*"troop" + 0.000*"sell" + 0.000*"library" + 0.000*"plan" + 0.000*"kitchen" + 0.000*"table" + 0.000*"college" + 0.000*"laughter" + 0.000*"girl" + 0.000*"shoot"'),
 (4,
  '0.067*"land" + 0.011*"principle" + 0.000*"troop" + 0.000*"silent" + 0.000*"object" + 0.000*"lie" + 0.000*"prayer" + 0.000*"shoe" + 0.000*"beacon" + 0.000*"saint"'),
 (5,
  '0.225*"people" + 0.022*"use" + 0.000*"troop" + 0.000*"silent" + 0.000*"object" + 0.000*"lie" + 0.000*"prayer" + 0.000*"shoe" + 

How to interpret this?

|Topic | representation
:-----:|:--------------:|
0 | '0.291*"thank" + 0.075*"make" + 0.059*"century" + 0.052*"strength" + 0.050*"first" + 0.017*"freedom" + 0.007*"do" + 0.001*"high" |
1 | '0.080*"take" + 0.005*"ceremony"
2 | 0.202*"time" + 0.000*"troop"
3 | ???
4 | 0.067*"land" + 0.011*"principle"
5 | 0.225*"people" + 0.022*"use"
6 | 0.336*"nation"
7 | 0.022*"protect" + 0.021*"bless" + 0.015*"story" + 0.013*"day" + 0.012*"world" + 0.011*"stand" + 0.011*"child" + 0.010*"let" + 0.010*"democracy" + 0.009*"know"
8 | ???
9 | 0.179*"today" + 0.158*"great" + 0.082*"year" + 0.017*"government" + 0.000*"troop"

<hr/>

## 14. Visualize the topics-keywords

Now that the LDA model is built, the next step is to examine the produced topics and the associated keywords. There is no better tool than pyLDAvis package’s interactive chart and is designed to work well with jupyter notebooks.

Keep in mind that [Topic 1 in pyLDAvis will be Topic 0 from gensim](
https://stackoverflow.com/a/62228381/653651).

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.367555 -0.018532       1        1  87.430836
0     -0.082299 -0.222307       2        1   3.016479
9     -0.063085  0.093595       3        1   2.466015
6     -0.050308  0.045277       4        1   1.437997
5     -0.041477  0.029723       5        1   1.387668
2     -0.037243  0.024902       6        1   1.195964
4     -0.026455  0.014336       7        1   0.877099
1     -0.027099  0.014922       8        1   0.865491
8     -0.019795  0.009043       9        1   0.670951
3     -0.019795  0.009043      10        1   0.651500, topic_info=             Term       Freq      Total Category  logprob  loglift
2           thank  90.000000  90.000000  Default  30.0000  30.0000
0          nation  50.000000  50.000000  Default  29.0000  29.0000
76          today  46.000000  46.000000  Default  28.0000  28.0000
103         great  40.000000  40.000000  Default  27.0000  27.0000
59         people  32.000000  32.000000  Default  26.0000  26.0000
..            ...        ...        ...      ...      ...      ...
23           hold   0.028165   1.086589  Topic10  -7.7761   1.3809
25      principle   0.028165   1.416058  Topic10  -7.7761   1.1161
26   inauguration   0.028165   0.523093  Topic10  -7.7761   2.1120
27          first   0.028165  15.887378  Topic10  -7.7761  -1.3015
28      president   0.028165  55.791748  Topic10  -7.7761  -2.5576

[592 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
746       1  0.904312     add
2377      1  1.014221    adda
2375      1  1.048013  anthem
667       1  0.959551  beacon
54        1  0.989760   begin
...     ...       ...     ...
1         1  0.823395    want
123       1  0.993415    work
212       1  0.999141   world
34        3  0.984297    year
2378      1  0.985850    youa

[106 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 10, 7, 6, 3, 5, 2, 9, 4])

## 15. How to interpret pyLDAvis’s output?

Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.


## 16. Compute Model Perplexity and Coherence Score

Model perplexity and [topic coherence](https://rare-technologies.com/what-is-topic-coherence/) provide a convenient measure to judge how good a given topic model is. In my experience, topic coherence score, in particular, has been more helpful.

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -9.857773449112718

Coherence Score:  0.6945382833043571


## 17. How to find the optimal number of topics for LDA?

The approach to finding the optimal number of topics is to build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value.

Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics.

If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

The `compute_coherence_values()` (see below) trains multiple LDA models and provides the models and their corresponding coherence scores.

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Can take a long time to run.
# Took 4.5 minutes last time
# More recently, threw AttributeError: module 'gensim.models' has no attribute 'wrappers'

# Commented out for now!
disable_cell = '''
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)
'''

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Show graph
# Commented out because coherence value could not be computed in the previous cell

disable_cell = '''
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()
'''

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Print the coherence scores
disable_cell = '''
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
'''

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


If the coherence score seems to keep increasing, it may make better sense to pick the model that gave the highest CV before flattening out. This is exactly the case here.

So for further steps I will choose the model with 12 topics.

In [ ]:
# Select the model and print the topics
disable_cell = '''
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=12))
'''

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Those were the topics for the chosen LDA model.

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 18. Finding the dominant topic in each sentence</h2>
</div>

One of the practical application of topic modeling is to determine what topic a given document is about.

To find that, we find the topic number that has the highest percentage contribution in that document.

The `format_topics_sentences()` function below nicely aggregates this information in a presentable table.

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


disable_cell = '''
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)
'''

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 19. Find the most representative document for each topic</h2>
</div>

Sometimes just the topic keywords may not be enough to make sense of what a topic is about. So, to help with understanding the topic, you can find the documents a given topic has contributed to the most and infer the topic by reading that document. Whew!!



In [ ]:
disable_cell = '''
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()
'''

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The tabular output above had 5 rows, one each for a topic. It has the topic number, the keywords and the most representative document. The `Perc_Contribution` column is nothing but the percentage contribution of the topic in the given document.

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 20. Topic distribution across documents</h2>
</div>

Finally, we want to understand the volume and distribution of topics in order to judge how widely it was discussed. The below table exposes that information.

In [ ]:
disable_cell = '''
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.head(5)
'''

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 21. Conclusion</h2>
</div>

We started with understanding what topic modeling can do. We built a basic topic model using Gensim’s LDA and visualize the topics using pyLDAvis. Then we built mallet’s LDA implementation. You saw how to find the optimal number of topics using coherence scores and how you can come to a logical understanding of how to choose the optimal model.

Finally we saw how to aggregate and present the results to generate insights that may be in a more actionable.

Hope you enjoyed reading this. I would appreciate if you leave your thoughts in the comments section below.

Edit: I see some of you are experiencing errors while using the LDA Mallet and I don’t have a solution for some of the issues. So, I’ve implemented a workaround and more useful [topic model visualizations](https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/). Hope you will find it helpful.

### All Rights Reserved. 

This notebook is proprietary content of machinelearningplus.com. This can be shared solely for educational purposes, with due credits to machinelearningplus.com

The original notebook was published by **Selva Prabhakaran** [online](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/).

There is an additional segment in Prabhakan's notebook that builds an LDA Mallet model. It has fallen prey to obsolescence &mdash; I really should take the time to fix it.